# This notebook is used for various network modifications such as:

* Adding centroids

## Starting up

Here the database is chosen, and scenarios/networks are being chosen

In [15]:
from pandas import *
import inro.emme.desktop.app as _app
my_app = _app.connect() #connecting to currently open EMME instance
my_modeller = inro.modeller.Modeller() #automatically connected to the current EMME instance
data_explorer = my_app.data_explorer()
database = data_explorer.active_database()
print database.title()

for scenario in database.scenarios():
    print scenario.title()

## Choose the scenario that the modifications will occur on

In [16]:
main_scenario = database.scenario_by_number(3001).core_scenario

The above scenario is copied to a backup (9999) before any modifications.

In [18]:
copy_scenario = my_modeller.tool("inro.emme.data.scenario.copy_scenario")

backup_scenario = copy_scenario(from_scenario=main_scenario,
                      scenario_id=9999,
                      scenario_title="Backup of scenario" + main_scenario.id,
                      copy_strategies=True,
                      copy_linkshapes=True,
                      overwrite=True)

modif_network = main_scenario.get_network()

## Point to the input excel file

The input file should be a .csv (comma-separated) with 3 columns in the folder *Scripts*, as follows (column names in first row):
* *Zone_ID*: The id number of the centroid
* *x_coord*: The x coordinate of the centroid in the EMME network coordinate reference system
* *y_coord*: The y coordinate of the centroid in the EMME network coordinate reference system

In [28]:
input_data = read_csv("test_nodes_add.csv")

In [33]:
zone_ids = input_data["Zone_ID"].tolist()
x_coords = input_data["x_coord"].tolist()
y_coords = input_data["y_coord"].tolist()

In [ ]:
for zone_id, x_coord, y_coord in zip(zone_ids, x_coords, y_coords):
    modif_network.create_node(str(zone_id), is_centroid = True)
    modif_network.node(str(zone_id)).x = x_coord
    modif_network.node(str(zone_id)).y = y_coord
    print "Node " + str(zone_id) + " created succesfully at " + str(x_coord) + ", " + str(y_coord) + "."
main_scenario.publish_network(modif_network)

## Delete nodes

Here you can define a list of node numbers, which will be deleted. If these nodes have any dependent links, an error is raised.

In [55]:
delete_nodes = [2000, 2001, 2002]

In [56]:
modif_network = main_scenario.get_network()
for node_id in delete_nodes:
    modif_network.delete_node(str(node_id), cascade = False) #cascade = True will delete node and all depended elements if they exist
main_scenario.publish_network(modif_network)